# Structural analysis model

> Creation of a truss model for structural analysis with IfcOpenShell

In [ ]:
# | default_exp _structural_analysis_model

In [ ]:
# | export

# Copyright © 2023-2024  IfcTruss Contributors
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [ ]:
# | hide
import nbdev
import nbdev.showdoc

The package ifcopenshell will be used to interact with IFC-SPF files. For more information about all the formats IFC has, head to this site:  
https://technical.buildingsmart.org/standards/ifc/ifc-formats/

For a tutorial about the ifcopenshell package, head to this site:  
https://blenderbim.org/docs-python/ifcopenshell-python/hello_world.html.

In [ ]:
# | export
from fastcore.basics import patch
import ifcopenshell
import numpy as np
import pandas as pd

In [ ]:
# | export
from ifctruss import __version__ as ifctruss_version
from ifctruss._truss_creation import TrussCreation

`StructuralAnalysisModelCreation` contains a method to create a basic ifc file and has a high-level method to create from a DataFrame with relevant information of a truss model the corresponding one in ifc.

In [ ]:
# | export
class StructuralAnalysisModelCreation:
    def __init__(self, version: str = "IFC4X3_ADD2"):
        self.ifc_model: ifcopenshell.file = ifcopenshell.file(
            schema=version
        )

        # IFC HEADER
        self.VIEW_DEFINITION: str = "ViewDefinition[]"

        # IfcProject
        self.PROJECT_NAME: str = "Structural analysis models"
        self.PROJECT_COORDINATE_SYSTEM = {
            "ORIGIN": (0.0, 0.0, 0.0),
            "LOCAL_Z_AXIS": (0.0, 0.0, 1.0),
            "LOCAL_X_AXIS": (1.0, 0.0, 0.0),
            "COORDINATE_SPACE_DIMENSION": 3,
            "PRECISION": 0.00001,
        }

        # IfcSIUnit
        self.SI_UNITS = (
            ("FORCEUNIT", "KILO", "NEWTON"),
            ("PRESSUREUNIT", "KILO", "PASCAL"),
            ("LENGTHUNIT", None, "METRE"),
            ("AREAUNIT", None, "SQUARE_METRE"),
        )

        # IfcStructuralAnalysisModels
        self.structural_analysis_models = []

Since there is no [MVD](https://technical.buildingsmart.org/standards/ifc/mvd/mvd-database/) currently for IFC4X3, the combination StructuralAnalysisView and DesignTransferView for the ViewDefinition is chosen. The [Implementation Guide for IFC Header Section](https://standards.buildingsmart.org/documents/Implementation/ImplementationGuide_IFCHeaderData_Version_1.0.2.pdf) species that there can be more than one ViewDefinition. Since there can be several structural analysis models in an ifc model, a list is created so that you have easy access to all of them.

In `create_basic_structure` a project and a context for the geometry is created, and the units are defined.

In [ ]:
# | export
@patch
def create_basic_structure(self: StructuralAnalysisModelCreation):
    # IFC Header
    self.ifc_model.wrapped_data.header.file_description.description = (
        self.VIEW_DEFINITION,
    )

    self.ifc_model.wrapped_data.header.file_name.originating_system = (
        f"IfcTruss v{ifctruss_version}"
    )

    # IFC Body
    ## Unit
    unit = []
    for SI_UNIT in self.SI_UNITS:
        unit.append(self.ifc_model.createIfcSIUnit(None, *SI_UNIT))
    units_in_context = self.ifc_model.createIfcUnitAssignment(unit)

    ## Project coordinate system
    origin = self.ifc_model.createIfcCartesianPoint(
        self.PROJECT_COORDINATE_SYSTEM["ORIGIN"]
    )
    z_axis = self.ifc_model.createIfcDirection(
        self.PROJECT_COORDINATE_SYSTEM["LOCAL_Z_AXIS"]
    )
    x_axis = self.ifc_model.createIfcDirection(
        self.PROJECT_COORDINATE_SYSTEM["LOCAL_X_AXIS"]
    )
    self.world_coordinate_system = self.ifc_model.createIfcAxis2Placement3D(
        origin, z_axis, x_axis
    )
    self.representation_contexts = (
        self.ifc_model.createIfcGeometricRepresentationContext(
            "3D",
            "Model",
            self.PROJECT_COORDINATE_SYSTEM["COORDINATE_SPACE_DIMENSION"],
            self.PROJECT_COORDINATE_SYSTEM["PRECISION"],
            self.world_coordinate_system,
            None,
        )
    )

    ## Project
    self.ifc_project = self.ifc_model.createIfcProject(
        ifcopenshell.guid.new(),
        None,
        self.PROJECT_NAME,
        None,
        None,
        None,
        None,
        [self.representation_contexts],
        units_in_context,
    )

In [ ]:
# | export
@patch
def create_from_DataFrame_a_truss_model(
    self: StructuralAnalysisModelCreation,
    *,
    Bars,
    Nodes,
    Point_Loads,
    Load_Groups,
):
    truss_model = TrussCreation(
        model=self.ifc_model,
        representation_contexts=self.representation_contexts,
        world_coordinate_system=self.world_coordinate_system,
        relating_context=self.ifc_project,
        Bars=Bars,
        Nodes=Nodes,
        Point_Loads=Point_Loads,
        Load_Groups=Load_Groups,
    )

    truss_model.create_structural_analysis_model()
    truss_model.create_nodes()
    truss_model.create_bars()
    truss_model.assignment_of_nodes_and_bars_to_the_analysis_model()
    truss_model.create_point_loads()
    truss_model.create_load_groups()
    truss_model.create_modulus_of_elasticitys()
    truss_model.create_surface_areas()
    truss_model.create_material_profiles()

    self.structural_analysis_models.append(truss_model)

In [ ]:
Nodes_data = {
    "Node": pd.Series([1, 2, 3, 4], dtype=int),
    "Coordinate_X": pd.Series([0, 0, -4e3, -4e3], dtype=float),
    "Coordinate_Y": pd.Series([0, 0, 0, 0], dtype=float),
    "Coordinate_Z": pd.Series([0, 3e3, 3e3, 6e3], dtype=float),
    "Translational_X": pd.Series([1, 0, 1, 1], dtype=bool),
    "Translational_Y": pd.Series([1, 0, 1, 1], dtype=bool),
    "Translational_Z": pd.Series([1, 0, 1, 1], dtype=bool),
}

Nodes = pd.DataFrame(Nodes_data)
Nodes.style.hide(axis="index")

Node,Coordinate_X,Coordinate_Y,Coordinate_Z,Translational_X,Translational_Y,Translational_Z
1,0.000000,0.000000,0.000000,True,True,True
2,0.000000,0.000000,3000.000000,False,False,False
3,-4000.000000,0.000000,3000.000000,True,True,True
4,-4000.000000,0.000000,6000.000000,True,True,True


In [ ]:
Bars_data = {
    "Bar": pd.Series([1, 2, 3], dtype=int),
    "Start_node": pd.Series([2, 2, 2], dtype=int),
    "End_node": pd.Series([1, 3, 4], dtype=int),
    "Cross-sectional_area": pd.Series([1e3, 1e3, 1e3], dtype=float),
    "Modulus_of_elasticity": pd.Series([1e3, 1e3, 1e3], dtype=float),
}
Bars = pd.DataFrame(Bars_data)
Bars.style.hide(axis="index")

Bar,Start_node,End_node,Cross-sectional_area,Modulus_of_elasticity
1,2,1,1000.000000,1000.000000
2,2,3,1000.000000,1000.000000
3,2,4,1000.000000,1000.000000


In [ ]:
Point_Loads_data = {
    "Point_Load": pd.Series(
        [
            1,
        ],
        dtype=int,
    ),
    "Node": pd.Series(
        [
            2,
        ],
        dtype=int,
    ),
    "Force_X": pd.Series(
        [
            100e3,
        ],
        dtype=float,
    ),
    "Force_Y": pd.Series(
        [
            0,
        ],
        dtype=float,
    ),
    "Force_Z": pd.Series(
        [
            -100e3,
        ],
        dtype=float,
    ),
}
Point_Loads = pd.DataFrame(Point_Loads_data)
Point_Loads.style.hide(axis="index")

Point_Load,Node,Force_X,Force_Y,Force_Z
1,2,100000.000000,0.000000,-100000.000000


In [ ]:
Load_Groups_data = {
    "Load_Group": pd.Series(
        [
            1,
        ],
        dtype=int,
    ),
    "Point_Load": pd.Series(
        [
            1,
        ],
        dtype=int,
    ),
}
Load_Groups = pd.DataFrame(Load_Groups_data)
Load_Groups.style.hide(axis="index")

Load_Group,Point_Load
1,1


In [ ]:
truss_test = StructuralAnalysisModelCreation()
truss_test.create_basic_structure()

In [ ]:
truss_test.create_from_DataFrame_a_truss_model(
    Bars=Bars, Nodes=Nodes, Point_Loads=Point_Loads, Load_Groups=Load_Groups
)

In [ ]:
truss_test.ifc_model.write("truss_excel_example.ifc")

In [ ]:
assert isinstance(truss_test.ifc_model, ifcopenshell.file)

In [ ]:
!cat "truss_excel_example.ifc"

ISO-10303-21;
HEADER;
FILE_DESCRIPTION(('ViewDefinition[]'),'2;1');
FILE_NAME('','2024-01-09T21:32:02',(),(),'IfcOpenShell v0.7.0-6c9e130ca','IfcTruss v0.1.0','');
FILE_SCHEMA(('IFC4X3_ADD2'));
ENDSEC;
DATA;
#1=IFCSIUNIT(*,.FORCEUNIT.,.KILO.,.NEWTON.);
#2=IFCSIUNIT(*,.PRESSUREUNIT.,.KILO.,.PASCAL.);
#3=IFCSIUNIT(*,.LENGTHUNIT.,$,.METRE.);
#4=IFCSIUNIT(*,.AREAUNIT.,$,.SQUARE_METRE.);
#5=IFCUNITASSIGNMENT((#1,#2,#3,#4));
#6=IFCCARTESIANPOINT((0.,0.,0.));
#7=IFCDIRECTION((0.,0.,1.));
#8=IFCDIRECTION((1.,0.,0.));
#9=IFCAXIS2PLACEMENT3D(#6,#7,#8);
#10=IFCGEOMETRICREPRESENTATIONCONTEXT('3D','Model',3,1.E-05,#9,$);
#11=IFCPROJECT('08fw6p4fzDbP_Gh8fassG5',$,'Structural analysis models',$,$,$,$,(#10),#5);
#12=IFCBOUNDARYNODECONDITION('Joint',IFCBOOLEAN(.T.),IFCBOOLEAN(.T.),IFCBOOLEAN(.T.),IFCBOOLEAN(.F.),IFCBOOLEAN(.F.),IFCBOOLEAN(.F.));
#13=IFCLOCALPLACEMENT($,#9);
#14=IFCSTRUCTURALANALYSISMODEL('2896RxNMrAewIjEG8KdTtn',$,'Truss model',$,$,.NOTDEFINED.,#9,(#66),$,#13);
#15=IFCRELDECLARES('1duT

Control the created IFC file with IfcOpenShell internal validation service.

In [ ]:
import ifcopenshell.validate
from rich import print

In [ ]:
json_logger = ifcopenshell.validate.json_logger()
ifcopenshell.validate.validate(truss_test.ifc_model, json_logger)
json_list = json_logger.statements
if json_list:
    for i in json_list:
        print(i)

An empty list will evaluate to False.

In [ ]:
assert not json_list

In [ ]:
# | hide
nbdev.nbdev_export()